In [3]:
import tensorflow as ts
import keras as ks
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
import scipy as sc
import sklearn as sk
import numpy as np
import pandas as pd
import os
from nltk.tokenize import sent_tokenize, word_tokenize 
from gensim.models import Word2Vec

Using TensorFlow backend.


In [6]:
train_path = os.path.join("C:\\Users\\ctg3039\\Downloads\\FolgerDigitalTexts_TXT_Complete\\", "1H4.txt")
sample = open(train_path, "r") 
s = sample.read()
data = [] 
#text = "\n".join([ll.rstrip() for ll in s.splitlines() if ll.strip()])
f = s.replace("\n", " ")
#Removing the non ASCII characters
#f = f[np.where(f < 128)]
# iterate through each sentence in the file 
for i in sent_tokenize(f):
    #print(i)
    temp = [] 
    # tokenize the sentence into words 
    for j in word_tokenize(i.strip()): 
        data.append(j.lower())
    #data.append(temp)

In [184]:
print(len((data)))
print(len(set(data)))
#print(len(sentences_test))
#print((sentences[2:4]))
#print((next_words[2:4]))
#print(len(next_words_test))

31675
3886


In [71]:
# Calculate word frequency
MIN_WORD_FREQUENCY = 2
word_freq = {}
for word in data:
    word_freq[word] = word_freq.get(word, 0) + 1

ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)

words = set(data)
print('Unique words before ignoring:', len(words))
print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_words)
print('Unique words after ignoring:', len(words))

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

#Creating the sequences for the sentences
STEP = 1
SEQUENCE_LEN = 10
sentences = []
next_words = []
ignored = 0
for i in range(0, len(data) - SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(data[i:i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(data[i: i + SEQUENCE_LEN])
        next_words.append(data[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))
#sentences, next_words, sentences_test, next_words_test = sk.model_selection.train_test_split(sentences, next_words)

Unique words before ignoring: 3886
Ignoring words with frequency < 2
Unique words after ignoring: 1650
Ignored sequences: 15849
Remaining sequences: 15816


In [9]:
#x = np.zeros((len(sentences), SEQUENCE_LEN, len(words)), dtype=np.bool)
#y = np.zeros((len(sentences), len(words)), dtype=np.bool)

In [10]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN, len(words)), dtype=np.bool)
        y = np.zeros((batch_size, len(words)), dtype=np.bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index]):
                x[i, t, word_indices[w]] = 1
            y[i, word_indices[next_word_list[index]]] = 1

            index = index + 1
            if index == len(sentence_list):
                index = 0
        yield x, y

In [11]:
model = Sequential()
model.add(ks.layers.Bidirectional(LSTM(128), input_shape=(SEQUENCE_LEN, len(words))))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
model.add(ks.layers.Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [30]:
BATCH_SIZE = 200
filename = "words-improvement-10-2.9530.hdf5"

checkpoint = ks.callbacks.ModelCheckpoint(filename, monitor='loss', verbose=1, save_best_only=True, mode='min')
#print_callback = ks.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = ks.callbacks.EarlyStopping(monitor='loss', patience=5)
callbacks_list = [checkpoint,early_stopping]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
    steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
    epochs=5,
    callbacks=callbacks_list)#,
    #validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1

Epoch 1/5
80/80 [==============================] - 38s 477ms/step - loss: 5.0766 - acc: 0.1141

Epoch 00001: loss improved from inf to 5.07656, saving model to words-improvement-10-2.9530.hdf5
Epoch 2/5
80/80 [==============================] - 34s 423ms/step - loss: 4.9600 - acc: 0.1219

Epoch 00002: loss improved from 5.07656 to 4.96001, saving model to words-improvement-10-2.9530.hdf5
Epoch 3/5
80/80 [==============================] - 33s 417ms/step - loss: 4.8688 - acc: 0.1321

Epoch 00003: loss improved from 4.96001 to 4.86880, saving model to words-improvement-10-2.9530.hdf5
Epoch 4/5
80/80 [==============================] - 34s 430ms/step - loss: 4.8023 - acc: 0.1389

Epoch 00004: loss improved from 4.86880 to 4.80234, saving model to words-improvement-10-2.9530.hdf5
Epoch 5/5
80/80 [==============================] - 37s 461ms/step - loss: 4.7313 - acc: 0.1449

Epoch 00005: loss improved from 4.80234 to 4.73131, saving model to words-improvement-10-2.9530.hdf5


In [72]:
# load the network weights
model = Sequential()
model.add(ks.layers.Bidirectional(LSTM(128), input_shape=(SEQUENCE_LEN, len(words))))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
model.add(ks.layers.Activation('softmax'))
filename = "words-improvement-10-2.9530.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [77]:
# pick a random seed
import sys
start = np.random.randint(0, len(sentences)-1)
print(start)
pattern = np.zeros((1, SEQUENCE_LEN, len(words)), dtype=np.bool)
for t, w in enumerate(sentences[start]):
    pattern[0,t, word_indices[w]] = 1

print("Seed:")
print("\"", ''.join(value for value in sentences[start]),"\"")
# generate characters
for i in range(100):
    prediction = model.predict(pattern, verbose=0)
    index = np.argmax(prediction)
    result = indices_word[index]
    sys.stdout.write(result)
    t = np.zeros((len(words)), dtype=np.bool)
    t[np.argmax(prediction)] = 1
    height = pattern[0].shape[0]
    pattern[0] = np.vstack([pattern[0], t])[1:(height+1),:]
print("\nDone.")

8848
Seed:
" faith.itellyouwhat--heheldme "
,iamalord,andiloweramiamaking.i'llnothim,iamalord,andiamaking.andi'llnotalord,andi'llbeafather.andi'llnotnotalord.i'llnotalord,andi'llnotafather.andi'llnotalord.andi'llnotnotalord.i'llnotalord,andi'llnotafather.andi'llnotalord
Done.


In [27]:
r = generator(sentences, next_words, 200)
print((next(r)[0].shape))
#print(x.shape)
#print(y.shape)

(200, 10, 1650)


In [64]:
start = np.random.randint(0, len(sentences)-1)
print(start)
pattern = np.zeros((1, SEQUENCE_LEN, len(words)), dtype=np.bool)
for t, w in enumerate(sentences[start]):
    pattern[0,t, word_indices[w]] = 1
prediction = model.predict(pattern, verbose=0)
indices_word[np.argmax(prediction)]

1221


'lord'

In [66]:
pattern.shape

(1, 10, 1650)

In [69]:
t = np.zeros((len(words)), dtype=np.bool)
t[np.argmax(prediction)] = 1
t

array([False, False, False, ..., False, False, False])

In [70]:
np.vstack([pattern[0], t]).shape

(11, 1650)